In [1]:
# jupyter notebook
#!jupyter nbextension enable --py --sys-prefix ipyleaflet

In [2]:
# jupyter lab
# !jupyter labextension install jupyter-leaflet
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [3]:
import os
import sys
import time
import pickle
import string
from datetime import datetime
from datetime import timedelta
from itertools import chain, cycle
import collections
from pprint import pprint
from io import StringIO
import pathlib

import numpy as np
import pandas as pd
from pandas.tools.plotting import parallel_coordinates
import matplotlib.pyplot as plt

#import urllib
#import webbrowser
#import codecs
from bs4 import BeautifulSoup
from xml.dom import minidom
from xml.etree import ElementTree as ET
#from lxml import etree as ET #Supports xpath syntax
import xmltodict 

import nltk
import re
import Levenshtein
import networkx as nx
import json

#from nominatim import Nominatim
#from mpl_toolkits.basemap import Basemap   #http://matplotlib.org/basemap/api/basemap_api.html
from matplotlib import cm
import geopy.geocoders as gg
from geopy.geocoders import Nominatim
import geopandas as gpd
import ipyleaflet
#from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, WMSLayer, Marker, MarkerCluster

##https://python-visualization.github.io/folium/docs-v0.6.0/
#https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf
import folium
import folium.plugins
#from folium.plugins import MarkerCluster
#from folium.plugins import FastMarkerCluster


#import plotly as py
#import plotly.graph_objs as go
#import plotly.figure_factory as ff

from IPython.display import Image, HTML, IFrame, FileLink, FileLinks #needed to render in notebook
from IPython.core.display import display

#import pydotplus #Install http://www.graphviz.org/ & Instal https://pypi.python.org/pypi/pydotplus

#https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary-in-python
#https://github.com/martinblech/xmltodict
#https://www.datacamp.com/community/tutorials/python-xml-elementtree
#https://stackabuse.com/reading-and-writing-xml-files-in-python/
#https://codereview.stackexchange.com/questions/117810/parsing-of-xml-in-python-using-minidom-and-inserting-data-into-a-database
#https://stackoverflow.com/questions/749796/pretty-printing-xml-in-python

In [4]:
pwd

'C:\\Users\\rsouza\\Documents\\Python Scripts\\exploreAT-Topothek.NLP'

#### Reading the XML Metadata files and processing Information on places

In [5]:
placesfile =  './TEI-XML-2018/helper_tables/listPlace.xml'
#print((open(placesfile, encoding="utf-8").read()))
placestree = ET.parse(placesfile)
placesroot = placestree.getroot()
set([elem.tag for elem in placesroot.iter()])

{'{http://www.tei-c.org/ns/1.0}Bundesland',
 '{http://www.tei-c.org/ns/1.0}Gemeinde',
 '{http://www.tei-c.org/ns/1.0}Großregion',
 '{http://www.tei-c.org/ns/1.0}Kleinregion',
 '{http://www.tei-c.org/ns/1.0}Kommentar',
 '{http://www.tei-c.org/ns/1.0}Ort',
 '{http://www.tei-c.org/ns/1.0}Sigle_DB',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}note',
 '{http://www.tei-c.org/ns/1.0}place',
 '{http://www.tei-c.org/ns/1.0}placeName',
 '{http://www.tei-c.org/ns/1.0}record'}

In [6]:
placesdict = xmltodict.parse((open(placesfile, encoding="utf-8").read()))
print(placesdict.keys())
print(placesdict['listPlace'].keys())

odict_keys(['listPlace'])
odict_keys(['@xmlns', 'place'])


In [7]:
placeslist = placesdict['listPlace']['place']
state_bundesland = [p['placeName'] for p in placeslist]
print(state_bundesland)

['W.', 'STir.', 'OTir.', 'NTir.', 'Kä.', 'St.', 'Sa.', 'OÖ', 'NÖ', 'Bgl.']


In [8]:
placeslist[0]['listPlace']['place']['listPlace']['place']['listPlace']['place']['listPlace']['place'][0]['record']
#placeslist[1]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['record']
#placeslist[2]['listPlace']['place']['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['record']
#placeslist[3]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['listPlace']['place']['record']

OrderedDict([('Ort', 'Altmannsdorf'),
             ('Gemeinde', 'Wien'),
             ('Kleinregion', 'W.'),
             ('Großregion', 'W.'),
             ('Bundesland', 'W.'),
             ('Sigle_DB', None),
             ('Kommentar', None)])

#### Using a geolocator to get the Coordinates

In [9]:
geolocator = gg.Nominatim(user_agent="test_application")

The next cells were used just for getting geographic information. The saved Pickle shall be used for the subsequent processing. 

In [34]:
df_places = pd.read_pickle("./df_places.pkl")
df_places.head(10)

,Bundesland,Ort,Coordinates,Adresses
0,W.,Altmannsdorf,"(10.4342455, 49.9358878)","Altmannsdorf, Gerolzhofen (VGem), Landkreis Sc..."
1,W.,Aspern,"(16.4847508, 48.2181854)","Aspern, Donaustadt, Wien, 1220, Österreich"
2,W.,Atzgersdorf,"(16.2954875, 48.1469838)","Atzgersdorf, Liesing, Wien, 1230, Österreich"
3,W.,Breitensee,"(10.5435061, 50.3531716)","Breitensee, Bad Königshofen i. Grabfeld (VGem)..."
4,W.,Favoriten,"(16.3828218364771, 48.1529937)","Favoriten, Wien, 1100, Österreich"
5,W.,Gersthof,None,None
6,W.,Hernals,"(16.2842142186391, 48.2354031)","Hernals, Wien, 1170, Österreich"
7,W.,Hietzing,"(16.2529858155656, 48.17854095)","Hietzing, Wien, 1130, Österreich"
8,W.,Inzersdorf,"(16.3527158778638, 48.14397215)","KG Inzersdorf, Liesing, Wien, 1230, Österreich"
9,W.,Kalksburg,"(16.2457624, 48.1376728)","Kalksburg, Liesing, Wien, 1230, Österreich"


Testing with a missing field...

Showing places in a map

In [35]:
df_places = pd.read_pickle("./df_places_plus.pkl")
df_places.head(10)

,Bundesland,Ort,Coordinates,Adresses
0,W.,Altmannsdorf,"(10.4342455, 49.9358878)","Altmannsdorf, Gerolzhofen (VGem), Landkreis Sc..."
1,W.,Aspern,"(16.4847508, 48.2181854)","Aspern, Donaustadt, Wien, 1220, Österreich"
2,W.,Atzgersdorf,"(16.2954875, 48.1469838)","Atzgersdorf, Liesing, Wien, 1230, Österreich"
3,W.,Breitensee,"(10.5435061, 50.3531716)","Breitensee, Bad Königshofen i. Grabfeld (VGem)..."
4,W.,Favoriten,"(16.3828218364771, 48.1529937)","Favoriten, Wien, 1100, Österreich"
5,W.,Gersthof,"(16.3237393, 48.2341382)","Gersthof, Währing, Wien, 1180, Österreich"
6,W.,Hernals,"(16.2842142186391, 48.2354031)","Hernals, Wien, 1170, Österreich"
7,W.,Hietzing,"(16.2529858155656, 48.17854095)","Hietzing, Wien, 1130, Österreich"
8,W.,Inzersdorf,"(16.3527158778638, 48.14397215)","KG Inzersdorf, Liesing, Wien, 1230, Österreich"
9,W.,Kalksburg,"(16.2457624, 48.1376728)","Kalksburg, Liesing, Wien, 1230, Österreich"


In [14]:
df_places.Coordinates[1][-1::-1]

(48.2181854, 16.4847508)

In [15]:
#https://ipyleaflet.readthedocs.io/en/latest/
#https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html
#https://github.com/jupyter-widgets/ipyleaflet/issues/64
#http://andrewgaidus.com/leaflet_webmaps_python/

#layer = WMSLayer(url="https://demo.boundlessgeo.com/geoserver/ows?", layers="nasa:bluemarble")
layer = ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.NASAGIBS.ModisTerraTrueColorCR, "2018-09-30")
center = (47,14)

mapplaces = ipyleaflet.Map(#layers=(layer,),
                            center=center,
                            zoom=6)

mapplaces.layout.width = '100%'
mapplaces.layout.height = '800px'

markers = []
for location in df_places.Coordinates:
    if location:
        markers.append(ipyleaflet.Marker(location=location[-1::-1], draggable=False,))
                       
marker_cluster = ipyleaflet.MarkerCluster(markers=tuple(markers))                       
mapplaces.add_layer(marker_cluster);


mapplaces

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [20]:
#using Folio

mapplaces = folium.Map(location=(47,14), zoom_start=4)

mc = folium.plugins.MarkerCluster()

for location in df_places.Coordinates:
    if location:
        mc.add_child(folium.Marker(location=location[-1::-1]))
                       
mapplaces.add_child(mc)
mapplaces

#### Reading the XML Data files:

In [11]:
xmlfile =  './TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml'
#xmlfile = open(xmlfile, mode='r', encoding='utf-8').read()
namespaces = {'TEI': 'http://www.tei-c.org/ns/1.0'} # add more as needed

Parsing the XML tree

In [12]:
xmldoc = minidom.parse(xmlfile)  #xmldoc = ET.XML(xmlfile)
bs = BeautifulSoup(open(xmlfile, encoding="utf8"), 'xml')
tree = ET.parse(xmlfile)

In [13]:
root = tree.getroot()
print(root)

<Element '{http://www.tei-c.org/ns/1.0}TEI' at 0x000001F423C1DAE8>


In [14]:
my_namespaces = dict([node for _, node in ET.iterparse(StringIO(open(xmlfile, mode='r', encoding='utf-8').read()), events=['start-ns'])])
print(my_namespaces)

{'': 'http://www.tei-c.org/ns/1.0'}


In [15]:
for child in root:
    print(child.tag, child.attrib)

{http://www.tei-c.org/ns/1.0}teiHeader {}
{http://www.tei-c.org/ns/1.0}text {}


In [16]:
set([elem.tag for elem in root.iter()])

{'{http://www.tei-c.org/ns/1.0}TEI',
 '{http://www.tei-c.org/ns/1.0}att',
 '{http://www.tei-c.org/ns/1.0}bibl',
 '{http://www.tei-c.org/ns/1.0}body',
 '{http://www.tei-c.org/ns/1.0}certainty',
 '{http://www.tei-c.org/ns/1.0}cit',
 '{http://www.tei-c.org/ns/1.0}date',
 '{http://www.tei-c.org/ns/1.0}def',
 '{http://www.tei-c.org/ns/1.0}edition',
 '{http://www.tei-c.org/ns/1.0}editionStmt',
 '{http://www.tei-c.org/ns/1.0}editorialDecl',
 '{http://www.tei-c.org/ns/1.0}encodingDesc',
 '{http://www.tei-c.org/ns/1.0}entry',
 '{http://www.tei-c.org/ns/1.0}etym',
 '{http://www.tei-c.org/ns/1.0}fileDesc',
 '{http://www.tei-c.org/ns/1.0}form',
 '{http://www.tei-c.org/ns/1.0}geoDecl',
 '{http://www.tei-c.org/ns/1.0}gi',
 '{http://www.tei-c.org/ns/1.0}gram',
 '{http://www.tei-c.org/ns/1.0}gramGrp',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}interp',
 '{http://www.tei-c.org/ns/1.0}label',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}listPrefixDe

In [17]:
places = []
for elem in root.iter('{http://www.tei-c.org/ns/1.0}place'):
    # print(elem.attrib)
    # print(elem.attrib['type'])
    places.append(elem.attrib['type'])
freq_places = collections.Counter(places)
freq_places.most_common(10)

[('Bundesland', 891),
 ('Großregion', 797),
 ('Kleinregion', 783),
 ('Gemeinde', 696),
 ('Ort', 696)]

Dictionary Approach

In [18]:
xml2dictfile = xmltodict.parse((open(xmlfile, mode='r', encoding='utf-8').read()))

In [19]:
print(xml2dictfile['TEI'].keys())

odict_keys(['@xmlns', 'teiHeader', 'text'])


In [20]:
xml2dictfile['TEI']['@xmlns']
#xml2dictfile['TEI']['teiHeader']

'http://www.tei-c.org/ns/1.0'

In [21]:
print(len(xml2dictfile['TEI']['text']['body']['entry']))
xml2dictfile['TEI']['text']['body']['entry'][0].keys()

1991


odict_keys(['@xml:id', '@xml:lang', 'form', 'gramGrp', 'sense', 'cit', 'note', 'ref', 'etym', 'usg'])

In [22]:
xml2dictfile['TEI']['text']['body']['entry'][14]

OrderedDict([('@xml:id', 'd157_qdb-d1e356'),
             ('@xml:lang', 'bar'),
             ('form',
              [OrderedDict([('@type', 'hauptlemma'), ('orth', 'Dinamo')]),
               OrderedDict([('@type', 'lautung'),
                            ('@n', '1'),
                            ('pron',
                             [OrderedDict([('@notation', 'tustep'),
                                           ('#text', 'DinEmon')]),
                              OrderedDict([('@notation', 'ipa'),
                                           ('@resp', '#JB'),
                                           ('@change', '01'),
                                           ('#text', 'DinEmon')])]),
                            ('gramGrp', OrderedDict([('gram', '[m,sg]')]))]),
               OrderedDict([('@type', 'nebenlemma'),
                            ('orth', 'Dinemon'),
                            ('gramGrp', OrderedDict([('pos', 'Subst')]))])]),
             ('gramGrp', OrderedDict([('pos',

In [23]:
datapath = './TEI-XML-2018/'
onlydirs = [f for f in os.listdir(datapath) if os.path.isdir(os.path.join(datapath,f)) and f.startswith('bel')]
onlydirs

['bel_d-tei-02',
 'bel_e-tei-02',
 'bel_f-tei-02',
 'bel_g-tei-02',
 'bel_h-tei-02',
 'bel_i-j-tei-02',
 'bel_k-tei-02',
 'bel_l-tei-02',
 'bel_m-tei-02',
 'bel_n-o-p-tei-02',
 'bel_q-r-tei-02',
 'bel_s-tei-02',
 'bel_u-w-tei-02',
 'bel_z-tei-02']

In [24]:
onlyfiles = []
for folder in onlydirs:
    for filex in [f for f in os.listdir(os.path.join(datapath, folder)) if os.path.isfile(os.path.join(datapath, folder, f)) \
                                       and not f.startswith('.')]:
        onlyfiles.append('{}{}/{}'.format(datapath, folder, filex))


print('Files in the folder:')
for i, w in enumerate(onlyfiles):
    print(i+1, '--' ,w)

Files in the folder:
1 -- ./TEI-XML-2018/bel_d-tei-02/d154_156_qdb-TEI-02.xml
2 -- ./TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml
3 -- ./TEI-XML-2018/bel_d-tei-02/d158_qdb-TEI-02.xml
4 -- ./TEI-XML-2018/bel_d-tei-02/d159_qdb-TEI-02.xml
5 -- ./TEI-XML-2018/bel_d-tei-02/d160_qdb-TEI-02.xml
6 -- ./TEI-XML-2018/bel_d-tei-02/d161_qdb-TEI-02.xml
7 -- ./TEI-XML-2018/bel_d-tei-02/d162_qdb-TEI-02.xml
8 -- ./TEI-XML-2018/bel_d-tei-02/d163_qdb-TEI-02.xml
9 -- ./TEI-XML-2018/bel_d-tei-02/d164_qdb-TEI-02.xml
10 -- ./TEI-XML-2018/bel_d-tei-02/d165_qdb-TEI-02.xml
11 -- ./TEI-XML-2018/bel_d-tei-02/d166_qdb-TEI-02.xml
12 -- ./TEI-XML-2018/bel_d-tei-02/d167_qdb-TEI-02.xml
13 -- ./TEI-XML-2018/bel_d-tei-02/d168_qdb-TEI-02.xml
14 -- ./TEI-XML-2018/bel_d-tei-02/d169_qdb-TEI-02.xml
15 -- ./TEI-XML-2018/bel_d-tei-02/d170_qdb-TEI-02.xml
16 -- ./TEI-XML-2018/bel_d-tei-02/d171_qdb-TEI-02.xml
17 -- ./TEI-XML-2018/bel_d-tei-02/d172_qdb-TEI-02.xml
18 -- ./TEI-XML-2018/bel_d-tei-02/d173_qdb-TEI-02.xml
19 -- ./TEI-

In [27]:
df_lemmas = pd.DataFrame(columns=['Ort', 'Lemma'])
orts = []
lemmas = []

for idx1, filex in enumerate(onlyfiles):
    xmldictfile = xmltodict.parse((open(filex, encoding="utf-8").read()))
    for idx2, entry in enumerate(xml2dictfile['TEI']['text']['body']['entry']):
        idx = ((idx1+1)*(idx2+1)-1)
        
        ort = None
        if 'usg' in entry.keys():
            if isinstance(entry['usg'], list):
                for local in entry['usg']:
                    try:
                        ort = local['placeName']['#text']
                    except:
                        ort = local['placeName']
            else:
                try:
                    ort = entry['usg']['placeName']['#text']
                except:
                    ort = entry['usg']['placeName']
        orts.append(ort)

        lemma = None
        if 'form' in entry.keys():
            if isinstance(entry['form'], list):
                for elem in entry['form']:
                    if elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], str):          
                        lemma = elem['orth']
                    elif elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], list):
                        for subelem in elem['orth']:
                            if subelem['@type'] == 'normalized':
                                lemma = subelem['#text']
            else:
                if isinstance(entry['form']['orth'], list):
                    for elem in entry['form']['orth']:
                        if elem['@type'] == 'normalized':
                            lemma = elem['#text']
                else:
                    lemma = entry['form']['orth']
        lemmas.append(lemma)
        
        if idx % 10000 == 0:
            print(idx)

df_lemmas['Ort'] = orts
df_lemmas['Lemma'] = lemmas

0
30000
20000
130000
10000
120000
110000
20000
170000
10000
200000
20000
40000
210000
120000
40000
350000
520000
20000
430000
430000
200000
610000
600000
860000
350000
890000
200000
300000
560000
980000
830000
830000
1030000
1070000
120000
870000
760000
590000
300000
980000
1200000
860000
920000
720000


In [28]:
df_lemmas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1451439 entries, 0 to 1451438
Data columns (total 2 columns):
Ort      798984 non-null object
Lemma    1451439 non-null object
dtypes: object(2)
memory usage: 22.1+ MB


In [30]:
df_lemmas.head(10)

,Ort,Lemma
0,Matrei/O. OTir.,Tin
1,Matrei/O. OTir.,Häutin
2,Matrei/O. OTir.,Gesotttin
3,Matrei/O. OTir.,Stadel
4,Matrei/O. OTir.,Tin
5,Matrei/O. OTir.,Stadeltor
6,Salurn,Tina
7,Salurn,Tina
8,(Umg.) Herzogenb-g. NÖ,Dinamít
9,None,Dinamít


In [38]:
df_lemmas = pd.read_pickle("./df_lemmas.pkl")
df_lemmas.head(10)

,Ort,Lemma
0,Matrei/O. OTir.,Tin
1,Matrei/O. OTir.,Häutin
2,Matrei/O. OTir.,Gesotttin
3,Matrei/O. OTir.,Stadel
4,Matrei/O. OTir.,Tin
5,Matrei/O. OTir.,Stadeltor
6,Salurn,Tina
7,Salurn,Tina
8,(Umg.) Herzogenb-g. NÖ,Dinamít
9,None,Dinamít


In [32]:
grouped = df_lemmas.groupby('Ort')['Lemma'].apply(list)

In [33]:
grouped.head()

Ort
"Passeier"                                       [dingen, dingen, dingen, Tinne, dingen, dingen...
"uWaldv." = öWaldv. in NÖ                        [Tinte, Tinte, Tinte, Tinte, Tinte, Tinte, Tin...
(D/Gm./Bz.) Poysdf. NÖ                           [Diras, Diras, Diras, Diras, Diras, Diras, Dir...
(Ebene Reichenau in Gm.) Reichenau Kä.           [Ding, Tingel Tangel, Tinte, sitzen, Piertippl...
(Leimsgrub u. Moarhäuser in Gm.) Schönbg. BöW    [Ding, Ding, Ding, Weile, Ding, Ding, hinterhi...
Name: Lemma, dtype: object

In [37]:
len(grouped)

428

#### Georeferencing Lemmas

In [ ]:
df_freqdist.Place.iloc[3].split()[1].strip(string.punctuation)

In [ ]:
geolocator = gg.Nominatim(user_agent="test_application")

In [ ]:
location = geolocator.geocode('salurn Österreich')
print(location.address)
address = location.address
print((location.latitude, location.longitude))
latitude, longitude = location.latitude, location.longitude
print(location.raw)

In [ ]:
def get_coordinates(place):
    location = None
    place = place.replace(')', '')
    place = place.replace('(', '')
    place = place.replace(']', '')
    place = place.replace('[', '')
    place = place.replace('.', '')
    place = place.replace('/', ' ')
    print(place)
    try:
        location = geolocator.geocode(place)
        if not location:
            try:
                location = geolocator.geocode(place.split(',')[0])
            except:
                print('failed 1')
        time.sleep(2)
        if not location:
            try:
                location = geolocator.geocode(place.split(',')[1])
            except:
                print('failed 2')
        time.sleep(2)
        if not location:
            try:
                location = geolocator.geocode(place.split(' ')[0])
            except:
                print('failed 3')
        time.sleep(2)
        if not location:
            try:
                location = geolocator.geocode(place.split(' ')[1])
            except:
                print('failed 4')
        time.sleep(2)
        if not location:
            try:
                location = geolocator.geocode(place.split(' ')[2])
            except:
                print('failed 5')
        if location:
            print('location -', location)
        else:
            print('place -', place)
    except:
        return None
    time.sleep(2)
    print('------')
    if location:
        return (location.longitude, location.latitude), location.address
    else:
        return None

In [ ]:
lemma_places = []
for p in df_freqdist.Place:
    lemma_places.append(get_coordinates(p))

In [ ]:
lemma_places

In [ ]:
idx = df_freqdist.Place.index
idx

In [ ]:
df_freqdist.loc[idx, 'Coordinates'] = [x[0] if x else None for x in lemma_places]

In [ ]:
df_freqdist.loc[idx, 'Adresses'] = [x[1] if x else None for x in lemma_places]

In [26]:
with open('./df_freqdist.pkl', 'rb') as f:
    df_freqdist = pickle.load(f)

In [27]:
df_freqdist.head()

,Place,Coordinates,Adresses
0,Matrei/O. OTir.,"(11.4489464, 47.122137)","Matrei, Brenner Autobahn, Mühlbachl, Innsbruck..."
1,Salurn,"(11.2110749, 46.2396083)","Salorno - Salurn, Überetsch-Unterland - Oltrad..."
2,(Umg.) Herzogenb-g. NÖ,"(23.3536536859833, 42.66048145)","МГУ, проф. Боян Каменов, район Студентски, Сто..."
3,Königswerth Egerld.,"(12.6803156, 50.1938078)","Královské Poříčí, okres Sokolov, Karlovarský k..."
4,Marling,"(11.1402909, 46.6555608)","Marling - Marlengo, Burggrafenamt - Burgraviat..."


In [28]:
def list2string(lista):
    texto = ''
    for elem in lista:
        texto = texto + elem[0] + ' --- ' + str(elem[1]) + '\n'
    return texto

In [31]:
print(list2string(freqdist[df_freqdist.loc[5,'Place']].most_common(5)))

tindern --- 1458
schebern --- 729
scheppern --- 729
tuschen --- 729



In [32]:
center = (47,14)
mapplaces = ipyleaflet.Map(#layers=(layer,),
                            center=center,
                            zoom=6)

mapplaces.layout.width = '100%'
mapplaces.layout.height = '800px'

markers = []
for idx, location in enumerate(df_freqdist.Coordinates):
    if (df_freqdist.loc[idx, 'Coordinates']) and (df_freqdist.loc[idx,'Place'] in freqdist.keys()):
        title = list2string(freqdist[df_freqdist.loc[idx,'Place']].most_common(6))
        markers.append(ipyleaflet.Marker(location=location[-1::-1], 
                              draggable=False, 
                              title=title))
                       
marker_cluster = ipyleaflet.MarkerCluster(markers=tuple(markers))                       
mapplaces.add_layer(marker_cluster);


mapplaces

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [33]:
#using Folio

mapplaces = folium.Map(location=(47,14),
               zoom_start=4)

mc = folium.plugins.MarkerCluster()

for idx, location in enumerate(df_freqdist.Coordinates):
    if (df_freqdist.loc[idx, 'Coordinates']) and (df_freqdist.loc[idx,'Place'] in freqdist.keys()):
        title = list2string(freqdist[df_freqdist.loc[idx,'Place']].most_common(6))
        mc.add_child(folium.Marker(location=location[-1::-1], popup=title))
                       
mapplaces.add_child(mc)
mapplaces